The exercise will focus on changing the `Activation Function`

In [1]:
import torch

In [2]:
# Define data
x_data = torch.tensor([[1.], [2.], [3.], [4.]], dtype=torch.float)
y_data = torch.tensor([[0.], [0.], [1.], [1.]], dtype=torch.float)

In [25]:
# 1. Design model class
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(1, 1)
#         self.activation_fn = torch.nn.Sigmoid()
#         self.activation_fn = torch.nn.Tanh() # RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got -0.146845 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THNN/generic/BCECriterion.c:62
#         self.activation_fn = torch.nn.Hardtanh() # RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got -1.000000 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THNN/generic/BCECriterion.c:62
#         self.activation_fn = torch.nn.Threshold(0.1, 1) # RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got 1.245499 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THNN/generic/BCECriterion.c:62
        self.activation_fn = torch.nn.LeakyReLU(-0.1) # RuntimeError: Assertion `x >= 0. && x <= 1.' failed. input value should be between 0~1, but got 1.205744 at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THNN/generic/BCECriterion.c:62

    def forward(self, x):
        x = self.linear(x)
        y_pred = self.activation_fn(x)
        return y_pred

In [26]:
# 2. Construct loss & optimizer
model = Model()
# criterion = torch.nn.BCELoss()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [27]:
# 3. Training
for t in range(1000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # Compute and print loss
    loss = criterion(y_pred, y_data)
#     print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [28]:
# After training
hour_var = torch.tensor([[1.]])
ans = bool(model.forward(hour_var).data[0][0] > 0.5)
print(f"predict 1 hour: {ans}")
assert False is ans

predict 1 hour: False


In [29]:
hour_var = torch.tensor([[7.]])
ans = bool(model.forward(hour_var).data[0][0] > 0.5)
print(f"predict 1 hour: {ans}")
assert True is ans

predict 1 hour: True


**BCELoss() + Sigmoid() -> Correct all**<br>
BCEWithLogitsLoss() + Sigmoid() -> 7 hours wrong prediction<br>
BCEWithLogitsLoss() + Tanh() -> 7 hours wrong prediction<br>
**BCEWithLogitsLoss() + Hardtanh() -> Correct all**<br>
BCEWithLogitsLoss() + Threshold(0.1, 1) -> 7hr wrong<br>
**BCEWithLogitsLoss() + LeakyReLU(-0.1) -> Correct all**<br>